# Evaluating Model Performance

In previous chapters, we evaluated our model performance by calling each estimator's `score` method which returned the value of $R^2$. We scored ourselves on the data that was used to train the model. Scoring models in this manner isn't indicative of future performance. Evaluating our model's performance on the same data that we trained it on will not give us a good measure of how likely it is to perform in the future.

This is akin to taking a test in school where the professor hands you the questions and answers beforehand. Our score gives us very little information about how well we would do on questions we've never encountered before.

### Estimate performance on unseen data

One of the major goals of machine learning is to have good knowledge of how well the model will perform on future unseen data. When the model is released into the wild, how will it perform? This is sometimes referred to as the [generalization error][1]. In other words, it is a measurement of how well the model generalizes to data that it has not seen before. There are several ideas that have been developed to calculate this generalization performance.

## First idea - split data into a training and test set

A simple idea involves partitioning the original dataset into two distinct datasets, one to be used during training, and another to be withheld for evaluating performance. The dataset not used during training is also referred to as the **test** dataset. Let's begin by reading in our data selecting three columns to be features in the model.

In [ ]:
import pandas as pd
housing = pd.read_csv('../data/housing_sample.csv')
X = housing[['GrLivArea', 'GarageArea', 'FullBath']]
y = housing['SalePrice']
X.head()

### Use the `train_test_split` helper function

The helper function `train_test_split` may be used to split the data into training and test sets. The `train_test_split` helper function is found in the `model_selection` module which contains many tools for evaluating model performance. It accepts both the input, `X`, and target variable, `y`. It splits each one into train and test sets returning the following four numpy arrays. 

* Input training set
* Input test set
* Output training set
* Output test set

Typically, more data is selected for the training than the test set. By default, 75% of the data will be selected for training with the remaining 25% selected for testing. You can set the proportion of data to be in the test set with the `test_size` parameter. By default, it is set to .25. 

You can also use the `train_size` parameter to set the proportion of data in the training set. If you provide exactly one of these parameters then, the other will equal its complement (so that the sum is equal to 1). Below, we set `test_size` equal to .3 which automatically makes the training size .7.

The observations for each set are chosen at random, so each additional time executing the function produces completely new data. However, you can set the `random_state` parameter to an integer which guarantees that the same random dataset is selected. Here we set it to 123, but any integer can work.

[1]: https://en.wikipedia.org/wiki/Generalization_error

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

The training/testing arrays need to have 70%/30% of the total rows. Let's compute these numbers now. The input arrays (`X`) need to have three columns. 

In [ ]:
int(.7 * len(X))

In [ ]:
int(.3 * len(X))

We unpacked the returned arrays into four separate variable names. Let's verify that all the arrays are the expected size. 

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

## Fit model just on the training data

To get an accurate measure of future performance, the model must never be exposed to the data in the test set. Below, we build a decision tree and fit the model on just the training data. Notice, that we do not set the maximum depth of the tree, which allows it to grow very deep.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train);

### Evaluating (incorrectly) on the training data

Before we evaluate on the test data, let's repeat our mistakes from the previous chapters and score ourselves on data that the model has already seen. Our $R^2$ value is extremely high and we nearly have a perfect fit.

In [ ]:
dtr.score(X_train, y_train)

### Evaluate on the test data

To more accurately assess our future performance, we evaluate on the test set, which is data that the model has never been exposed to.

In [ ]:
dtr.score(X_test, y_test)

### Overfitting

Our performance on the test set decreased dramatically from the results on the training set. We call it **overfitting** whenever a model performs much worse on the the test set than on the training set. This is similar to memorizing all the answers to previous practice exams and then failing during the actual final exam. Overfitting will be discussed in detail later.

## Next Idea - Cross Validation

One issue with the first idea above is that that there is only a single test set to evaluate our performance. If this particular test set, by chance, was not a good representation of the data it might not accurately measure our generalization performance. 

**Cross Validation** is a set of ideas that improves upon evaluation based on a single test set. Instead, we make many repetitive train/test splits of our data and record a score for each split. In this manner, we can calculate multiple performance scores, giving us more feedback on what to expect from unseen data.

### Many cross validation strategies

There are multiple strategies for splitting the data that fall under the umbrella of cross validation. By far, the most common form of cross validation is **K-fold cross validation**. In this form, the data is split into k distinct partitions, where k is usually between 5 and 10. One of these k partitions is used as the test set, while the other k-1 partitions are used for training. The model is fit on the data in these k-1 partitions. It is then evaluated on the one partition not used during training. This score is recorded and ends the first round of cross validation.

After the first round of testing, a different partition is used for testing and the other k-1 partitions are again used for training. The model is refit to the new training data and evaluated on the new test data. After all k partitions have been used for testing, cross validation ends.

The result is a total of k scores, which can then be averaged to yield a better overall performance metric. Take a look at the image below to see how the data would be split when doing a 4-fold cross validation.

![][1]

## K-Fold Cross Validation in Scikit-Learn

The `cross_val_score` function automates the process of doing cross validation for us in a single line. Pass it an estimator, the data, and the number of validation sets to use. The estimator does NOT need to be trained ahead of time as the `cross_val_score` function will re-train it during each iteration. Below, we use 5 folds (splits), meaning 80% of the data is used for training and 20% for testing during each round.

[1]: images/kfold.png

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

Now, that we've imported and instantiated our model, we can pass it to the `cross_val_score` function to automate the splitting and evaluation on five different folds. The `cv` parameter controls the number of folds.

In [ ]:
scores = cross_val_score(lr, X, y, cv=5)
scores.round(2)

### An array of scores is returned

An array of k scores is returned. We can take the mean and standard deviation to get a summary of our results.

In [ ]:
scores.mean()

In [ ]:
scores.std()

### Cross validation evaluates future performance

Cross validation is a procedure to inform us approximately how the model will perform in the future on unseen data. This is the major purpose of machine learning - to be able to predict future events. We care about future performance and cross validation is a procedure to estimate it.

### How well will we do on our college test?

Imagine you are a student in college preparing for a test and have a goal of achieving a high score. Before you take the actual test, you'd like to know your performance on it. One way to estimate your future performance is to find old test questions (that would be similar to the ones on your test) and answer them in the exact same setting as you would with a real test. Scoring ourselves in this manner would approximate our performance on the real test.

### Reporting our generalization performance estimate

In machine learning, cross validation is the procedure for producing scores that approximate performance on unseen data. It's important to summarize the information produced from cross validation. In this example, our linear regression instance, `lr`, achieved an average $R^2$ of .61 with a standard deviation of .07.

## Other flavors of cross validation

There are several other [cross validation strategies][1] that scikit-learn offers and refers to them as [splitter classes][2].

### Splitter classes are not estimators

Most of the scikit-learn API can be divided between estimators and helper functions. The splitter classes are a special case that don't fall into one of those categories. These objects need to be instantiated but they are not machine learning models. They are simpler objects used for cross validation.

### The `KFold` splitter

When we first called `cross_val_score`, we set the `cv` parameter to an integer, which was used to determine the number of splits. By default, `cross_val_score` does k-fold cross validation with five splits.

Instead of setting `cv` to an integer, we can set it to an instance of one of the splitter classes for a more customized type of split. All of the splitter classes are found in the `model_selection` module. Let's import the `KFold` splitter class and instantiate it so that it does 5 splits by setting the `n_splits` parameter.

[1]: https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators
[2]: https://scikit-learn.org/stable/modules/classes.html#splitter-classes

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

### No cross validation has happened at this point

The above command does not do any cross validation. The splitter `kf` is created which can be passed to the `cross_val_score` function to control the type of split done. Let's see this now.

In [ ]:
scores = cross_val_score(lr, X, y, cv=kf)
scores.round(2)

Notice how the scores are the exact same as the previous call to `cross_val_score` with the `cv` parameter set to 5. By default, `cross_val_score` does k-fold cross validation.

### Shuffling the data

By default, scikit-learn splits the dataset from the beginning in the order that it was received. It does not select the data at random. If the dataset is ordered in some way, this might affect the results. The model might not get exposure to observations from a particular segment of the data and therefore be unable to learn from it. To handle this issue, set the `shuffle` parameter to `True`. It is available in many of the splitter classes. Each time we run the following cell, we'll get different scores as the data is shuffled before the partitions are created.

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
cross_val_score(lr, X, y, cv=kf).round(2)

## Cross validation on other models

Cross validation can help us determine which models perform best on unseen data. We can build multiple models and then use the cross-validated mean score to rank them. Before, we do this, we need to ensure that the randomly shuffled data is the same for each model so that we are comparing scores on the same test data. The `random_state` parameter of the splitter classes may be set to ensure that the splits happen in the same random order each time.

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)

Let's get the mean cross-validated scores for the models linear regression, k-nearest neighbors, and decision trees.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
lr = LinearRegression()
knr = KNeighborsRegressor()
dtr = DecisionTreeRegressor()

scores = {}
scores['lr'] = cross_val_score(lr, X, y, cv=kf).mean().round(3)
scores['knr'] = cross_val_score(knr, X, y, cv=kf).mean().round(3)
scores['dtr'] = cross_val_score(dtr, X, y, cv=kf).mean().round(3)
scores

Using these exact models, with these three features, our cross-validated results show us that linear regression and k-nearest neighbors dramatically outperform decision trees. With this information, we can choose a model to use going forward to predict sale price of new houses that come on the market. Since linear regression did have have the best performance, we could choose that one. In upcoming chapters, we will see how to construct other models so they have better performance. Thus, the results above are subject to change.

## No model is returned

When using the `cross_val_score` function for k-fold cross validation, k models are trained. All of these models are different, as they are fit on different partitions of the data. None of those fitted models are returned to you. Only the scores are returned. You can verify that the model isn't trained by training to make a prediction with it. You'll raise the `NotFittedError`, meaning that the `fit` method has yet to be called.

In [ ]:
lr.predict(X)

### Fit on all of the data

In order to get a trained model that is ready to make predictions, you must call the `fit` method on it with all of the data.

In [ ]:
lr.fit(X, y)

When new data comes in, this model would be used to make predictions. You expect the $R^2$ on future performance to be around .61.

### Final Report

If you were reporting your findings at a meeting, you would say something like this: We expect the linear regression model fit on the features ground living area, garage area, and full baths to achieve an $R^2$ of about .61 when used to predict sale prices of new houses that come on the market. The linear regression model achieved a higher future performance estimate than k-nearest neighbors and decision trees.

## Exercises

### Exercise 1
<span  style="color:green; font-size:16px">Practice using the `ShuffleSplit`, `RepeatedKFold`, and `LeavePOut` splitters for any of the estimators we have covered. [Read the documentation and find out how they work][1].</span>

[1]: https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators